In [1]:
import tensorflow as tf

!pip install mitdeeplearning
import mitdeeplearning as mdl

import numpy as np
import os
import time
import functools
from IPython import display as ipythondisplay
from tqdm import tqdm

In [2]:
songs = mdl.lab1.load_training_data()

example_songs = songs[0]
print("\nExample song:")
print(example_songs)

Found 816 songs in text

Example song:
X:2
T:An Buachaill Dreoite
Z: id:dc-hornpipe-2
M:C|
L:1/8
K:G Major
GF|DGGB d2GB|d2GF Gc (3AGF|DGGB d2GB|dBcA F2GF|!
DGGB d2GF|DGGF G2Ge|fgaf gbag|fdcA G2:|!
GA|B2BG c2cA|d2GF G2GA|B2BG c2cA|d2DE F2GA|!
B2BG c2cA|d^cde f2 (3def|g2gf gbag|fdcA G2:|!


In [3]:
mdl.lab1.play_song(example_songs)

In [4]:
songs_joined = "\n\n".join(songs) 

In [5]:
vocab = sorted(set(songs_joined))
print("There are", len(vocab), "unique characters in the dataset")

There are 83 unique characters in the dataset


In [6]:
char2idx = {u:i for i,u in enumerate(vocab)}
idx2char = np.array(vocab)

In [7]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char),char2idx[char]))
print('   ...\n')

{
  &#39;\n&#39;:   0,
  &#39; &#39; :   1,
  &#39;!&#39; :   2,
  &#39;&quot;&#39; :   3,
  &#39;#&#39; :   4,
  &quot;&#39;&quot; :   5,
  &#39;(&#39; :   6,
  &#39;)&#39; :   7,
  &#39;,&#39; :   8,
  &#39;-&#39; :   9,
  &#39;.&#39; :  10,
  &#39;/&#39; :  11,
  &#39;0&#39; :  12,
  &#39;1&#39; :  13,
  &#39;2&#39; :  14,
  &#39;3&#39; :  15,
  &#39;4&#39; :  16,
  &#39;5&#39; :  17,
  &#39;6&#39; :  18,
  &#39;7&#39; :  19,
   ...



In [8]:
def vectorize_string(string):
    vectorized_output = np.array([char2idx[char] for char in string])
    return vectorized_output


In [9]:
vectorized_songs = vectorize_string(songs_joined)

In [10]:
print ('{} ---- characters mapped to int ----> {}'.format(repr(songs_joined[:10]), vectorized_songs[:10]))
# check that vectorized_songs is a numpy array
assert isinstance(vectorized_songs, np.ndarray), "returned result should be a numpy array"

&#39;X:2\nT:An B&#39; ---- characters mapped to int ----&gt; [49 22 14  0 45 22 26 69  1 27]


In [59]:
def get_batch(vectorized_songs, seq_length, batch_size):
    n = vectorized_songs.shape[0]-1
    idx = np.random.choice(n-seq_length, batch_size)

    input_batch = [vectorized_songs[i:i+seq_length] for i in idx]
    output_batch = [vectorized_songs[i+1: i+seq_length+1] for i in idx]

    x_batch = np.reshape(input_batch,[batch_size, seq_length])
    y_batch = np.reshape(output_batch, [batch_size, seq_length])
    return x_batch, y_batch

test_args = (vectorized_songs, 10, 2)
if not mdl.lab1.test_batch_func_types(get_batch, test_args) or \
   not mdl.lab1.test_batch_func_shapes(get_batch, test_args) or \
   not mdl.lab1.test_batch_func_next_step(get_batch, test_args): 
   print("======\n[FAIL] could not pass tests")
else: 
   print("======\n[PASS] passed all tests!")

[PASS] test_batch_func_types
[PASS] test_batch_func_shapes
[PASS] test_batch_func_next_step
[PASS] passed all tests!


In [61]:
x_batch, y_batch = get_batch(vectorized_songs, seq_length=5, batch_size=1)

for i, (input_idx, target_idx) in enumerate(zip(np.squeeze(x_batch), np.squeeze(y_batch))):
    print("Step {:3d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step   0
  input: 11 (&#39;/&#39;)
  expected output: 20 (&#39;8&#39;)
Step   1
  input: 20 (&#39;8&#39;)
  expected output: 0 (&#39;\n&#39;)
Step   2
  input: 0 (&#39;\n&#39;)
  expected output: 36 (&#39;K&#39;)
Step   3
  input: 36 (&#39;K&#39;)
  expected output: 22 (&#39;:&#39;)
Step   4
  input: 22 (&#39;:&#39;)
  expected output: 29 (&#39;D&#39;)


In [62]:
def LSTM(rnn_units):
    return tf.keras.layers.LSTM(
        rnn_units,
        return_sequences=True,
        recurrent_initializer='glorot_uniform',
        recurrent_activation='sigmoid',
        stateful=True,
    )

In [63]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size,None]),
        LSTM(rnn_units),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

model = build_model(len(vocab), embedding_dim=256,rnn_units=1024, batch_size=32)

In [64]:
model.summary()

Model: &quot;sequential&quot;
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (32, None, 256)           21248     
_________________________________________________________________
lstm (LSTM)                  (32, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (32, None, 83)            85075     
Total params: 5,353,299
Trainable params: 5,353,299
Non-trainable params: 0
_________________________________________________________________
